# **1. Importação das bibliotecas e dos dados**

A seção 1 foi destinada para a implementação das bibliotecas e também os arquivo csv que está no diretório do google drive (pelo google.colab). Destando-se a biblioteca do sklear a qual foram importadas os modelos de regressao logistica e árovre de decisão.

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive
import seaborn as sns

In [ ]:
#baixando os recursos do NLTK necessários:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#conexao com o google drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#leitura da base de dados na pasta treinamento_PLN
base_dados = pd.read_csv(
    "/content/drive/MyDrive/treinamento_PLN/imdb-reviews-pt-br.csv",
    engine="python",
    encoding="utf-8"
)

# **2. Pré-processamento dos dados**

In [ ]:
#exibindo a base de dados para termos conhecimento
base_dados.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [ ]:
#como a base de dados contém dados em inglês e não é do nosso interesse, temos que excluir as ID e text_en
base_dados.drop(["id", "text_en"],
          axis=1,
          inplace=True)

In [ ]:
base_dados.head()

,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,Este é um exemplo do motivo pelo qual a maiori...,neg
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,Filmes de fotos de latão não é uma palavra apr...,neg


In [ ]:
#como visto na tabela acima, os dados de sentimento são neg (referindo-se a negativo) e pos (para comentários positivos)
#É preciso fazer um tratamento para que os dados sejam interpretados por 1 (positivo) e 0 (negativo), logo:
base_dados['sentiment'] = base_dados['sentiment'].replace({'pos': 1, 'neg': 0})

In [ ]:
base_dados.head()

,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",0
1,Este é um exemplo do motivo pelo qual a maiori...,0
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",0
3,Nem mesmo os Beatles puderam escrever músicas ...,0
4,Filmes de fotos de latão não é uma palavra apr...,0


In [ ]:
base_dados.tail()

,text_pt,sentiment
49454,"Como a média de votos era muito baixa, e o fat...",1
49455,O enredo teve algumas reviravoltas infelizes e...,1
49456,Estou espantado com a forma como este filme e ...,1
49457,A Christmas Together realmente veio antes do m...,1
49458,O drama romântico da classe trabalhadora do di...,1


In [ ]:
# Como visto acima, os dados estao agrupados noo inicio como negativos e no fim como positivo.
# Para um melhor resultados modelos é preciso embaralhar os dados e gerar uma aleatoriedade, logo:
base_dados = base_dados.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
#resultado da base de dados misturada
base_dados.head()

,text_pt,sentiment
0,Isso era incomum: um filme moderno que era ult...,1
1,Alguns dos meus velhos amigos sugeriram que eu...,0
2,Que prazer. Isto é realmente uma paródia. Some...,1
3,"Há cerca de dez minutos a meio da Strangeland,...",0
4,"Otelo, a clássica história de Shakespearen sob...",1


# **3. Tokenizacao**

No tópico 3 se faz o processo de tokenizacao, onde cada palavra ou caractere do texto, no nosso caso dos comentários da base de dados, são serão dividos em tokens. Para isso, foi utilizado a "word_tokenize" da biblioteca NLTK para realizar o processo.

In [ ]:
# Tokenização dos comentários
stop_words = set(stopwords.words('portuguese'))

In [ ]:
def tokenize_text(text):
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(words)

In [ ]:
#aplicando a função tokenize_text para cada elemento de text_pt e criando uma nova coluna que receba a aplicação da função (contendo os tokens).
base_dados['tokenized_text'] = base_dados['text_pt'].apply(tokenize_text)

# **4. Treinamento e avaliação**

TF-IDF foi a técnica usada, pois é amplamente usada em PLN por ter importância relativa de palavras em um documento dentro de uma coleção de documentos e muito utlizada na classificação de textos.

In [ ]:
# Dividindo os dados em conjuntos de treinamento e teste: 20% estao destinado ao teste
X = base_dados['tokenized_text']
y = base_dados['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Criando a matriz TF-IDF para representar os dados:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
#Agora, o código abaixo treina o modelo de regressao logistica
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [ ]:
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy}")

Acurácia do modelo: 0.8831378892033966


In [ ]:
def preprocess_text(text):
    tokenized_text = tokenize_text(text)
    return vectorizer.transform([tokenized_text])

In [ ]:
def predict_sentiment(text):
    preprocessed_text = preprocess_text(text)
    prediction = model.predict(preprocessed_text)
    return 'positivo' if prediction[0] == 1 else 'negativo'

In [ ]:
# Checagem do resultado, escrevendo um texto qualquer sobre o filmes...
texto_usuario = "impressionante!"
sentimento = predict_sentiment(texto_usuario)
print(f"O sentimento do texto é: {sentimento}")

O sentimento do texto é: positivo


# **5. Comparativo e conclusões**

Também foi calculado a acurácia pela **Árvore de Decisão** para ser feito um comparativo. A princípio pensei que o código que está comentado seria necessário, mas para otimizar não é necessário.


In [ ]:
#bd = base_dados

In [ ]:
# Tokenização dos comentários
#stop_words = set(stopwords.words('portuguese'))

In [ ]:
#def tokenize_text(text):
#    words = word_tokenize(text.lower())
#    words = [word for word in words if word.isalpha() and word not in stop_words]
#    return ' '.join(words)

In [ ]:
#bd['tokenized_text'] = bd['text_pt'].apply(tokenize_text)

In [ ]:
#X = bd['tokenized_text']
#y = bd['sentiment']
#
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#vectorizer = TfidfVectorizer(max_features=5000)
#X_train_tfidf = vectorizer.fit_transform(X_train)
#X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train_tfidf, y_train)

DecisionTreeClassifier(random_state=42)

In [ ]:
y_pred_dt = dt_model.predict(X_test_tfidf)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"Acurácia do modelo Árvore de Decisão: {accuracy_dt}")

Acurácia do modelo Árvore de Decisão: 0.70380105135463


In [ ]:
#exibindo a base com o processamento e feito, incluindo a coluna tokanized_text
base_dados.head()

,text_pt,sentiment,tokenized_text
0,Isso era incomum: um filme moderno que era ult...,1,incomum filme moderno verdade tão bom difícil ...
1,Alguns dos meus velhos amigos sugeriram que eu...,0,alguns velhos amigos sugeriram assistisse film...
2,Que prazer. Isto é realmente uma paródia. Some...,1,prazer realmente paródia somente franceses pod...
3,"Há cerca de dez minutos a meio da Strangeland,...",0,cerca dez minutos meio strangeland repente vê ...
4,"Otelo, a clássica história de Shakespearen sob...",1,otelo clássica história shakespearen sobre amo...


In [ ]:
print(f"Acurácia do modelo de Regressao Logistica: {accuracy}")
print(f"Acurácia do modelo Árvore de Decisão: {accuracy_dt}")

Acurácia do modelo de Regressao Logistica: 0.8831378892033966
Acurácia do modelo Árvore de Decisão: 0.70380105135463


In [ ]:
print(f"Acurácia do modelo de Regressão Logística: {round(accuracy * 100, 2)}%")
print(f"Acurácia do modelo Árvore de Decisão: {round(accuracy_dt * 100, 2)}%")

Acurácia do modelo de Regressão Logística: 88.31%
Acurácia do modelo Árvore de Decisão: 70.38%


O trabalho desenvolvido buscou desenvolver modelos de machine learning que
foram apresentados nas aulas e trata-se de um processamento textual. A base de dados não teve relação com o assunto do TRE, entretanto, a abordagem e o estudo desses modelos podem complementar o trabalho dos residentes e também trazer avanços nos estudos de ML.
Como resultado do código acima, utlizando uma base de dados que trata de comentário de filmes e o sentimentos relacionados aos comentários, foi possível aplicar os modelos e por fim fazer um comparativo entre ambos. A  Regrecao logística obteu um resuldado de 88,31%, enquanto a Árvore de decisão de 70,38%.
Vários testes foram feitos colocanto comentários aleatórios para checagem do resultado.
O trabalho anteriormente feito, utilizando BERT e tokenizacao das palavras em inglês, foi muito importante para o aprendizado, ambos trabalhos abordam o carregamento, pré-processamento, treinamento e teste.

Obrigado
